In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import lightgbm as lgb
import multiprocessing as mul 
# BUG点：from multiprocessing import Pool 这里只能用 mul.Pool，不然系统会误以为调用caboost里面的Pool       
warnings.filterwarnings('ignore')

fea_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/fea_pkl/'
model_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/model_pkl/'
datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
fea_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Features/'
pred_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/'
sub_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/sub/'

BASE = fea_pkl_path + 'grid_part_1.pkl'
PRICE = fea_pkl_path + 'grid_part_2.pkl'
CALENDAR = fea_pkl_path + 'grid_part_3.pkl'
LAGS = fea_pkl_path + 'grid_part_4.pkl'
MEAN_ENC = fea_pkl_path + 'grid_part_5.pkl'

In [ ]:
## 设定随机种子
def seed_everything(seed=0):
  random.seed(seed)
  np.random.seed(seed)

## 按照商店读取数据
def get_data_by_store(store):
    # 读取和连接基本特征
    df = pd.concat([pd.read_pickle(BASE), # 9
             pd.read_pickle(PRICE).iloc[:,2:], # 11
             pd.read_pickle(CALENDAR).iloc[:,2:], # 15 
             pd.read_pickle(LAGS).iloc[:,3:], # 37
             pd.read_pickle(MEAN_ENC)[mean_features]], # 6  
             axis=1) 
    # df：(47735397, 78)由于行数相同，可以横向拼接
    df = df[df['store_id'] == store] # df['store_id'] == store 返回布尔Series，df[df['store_id'] == store] 然后取True的行构成的dataframe
    features = [col for col in list(df) if col not in remove_features] # 78 - 7 = 71
    df = df[['id', 'd', TARGET] + features] # 71 + 3 = 74 相当于去掉了 'state_id', 'store_id', 'release', 'Holiday'(在CALENDAR里面)
    # 选训练开始的天数
    df = df[df['d'] >= START_TRAIN].reset_index(drop=True) # drop = True去掉生成的原索引列
    return df, features


## 读取测试数据
def get_base_test():
  base_test = pd.DataFrame()
  
  if USE_AUX:
    model_path = model_pkl_path
  else:
    model_path=''

  for store_id in STORES_IDS: 
    temp_df = pd.read_pickle(model_path + 'cb_test_{}.pkl'.format(store_id)) # 不含lag + rolling特征
    temp_df['store_id'] = store_id
    base_test = pd.concat([base_test, temp_df]).reset_index(drop=True) # 纵向拼接
  
  return base_test
## 制作lag特征
def make_lag(LAG_DAY):
  lag_df = base_test[['id','d',TARGET]]
  col_name = 'sales_lag_' + str(LAG_DAY)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
  return lag_df[[col_name]]
## 递归特征
def make_lag_roll(LAG_DAY):
  shift_day = LAG_DAY[0]
  roll_wind = LAG_DAY[1]
  lag_df = base_test[['id','d',TARGET]]
  col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
  return lag_df[[col_name]]
## 多线程执行，用于测试集融合
def df_parallelize_run(func, t_split):
  num_cores = np.min([N_CORES,len(t_split)])
  pool = mul.Pool(num_cores)
  df = pd.concat(pool.map(func, t_split), axis=1)
  pool.close()
  pool.join()
  return df

# lag + rolling
SHIFT_DAY = 28
N_LAGS = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY, SHIFT_DAY + N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
  for j in [7,14,30,60]:
    ROLS_SPLIT.append([i,j])

VER = 1 # 设置模型的版本
SEED = 42 
seed_everything(SEED) # 消除随机性
N_CORES = psutil.cpu_count() # 可使用的CPU内核

TARGET = 'sales' # Label
START_TRAIN = 0 # 能跳过一些行(Nans/faster training)
P_HORIZON = 28 # 预测范围
USE_AUX = True # 使用预训练好的模型

remove_features = ['id', 'state_id', 'store_id', 'release', 'Holiday', 'd', TARGET]          
mean_features = ['enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std'] 
# 按商店分别训练，每个商店可以只能按类别、部门、商品的销量聚合取mean\std，故只选这6个特征

## 每个商店的cb参数

In [ ]:
!pip install catboost
from catboost import Pool, CatBoostRegressor 
# Pool是catboost中的用于组织数据的一种形式，也可以用numpy、array和dataframe，但更推荐Pool，其内存和速度都更优。
cb_params_common = { 
            'iterations' : 1524,
            'learning_rate' : 0.08868565834866991,
            'l2_leaf_reg' : 9.398124479877186,    
            'subsample' : 0.786596695482429,
            'max_depth' : 7,
            'boosting_type' : 'Plain',
            'one_hot_max_size' : 2,
            'max_bin' : 142,
            'objective' : 'RMSE',
            'eval_metric' : 'RMSE',
            'random_seed' : 42,
            'leaf_estimation_method' : 'Gradient',
            'nan_mode' : 'Max',
            'min_data_in_leaf' : 11965,
            'boost_from_average' : False,
            'verbose' : 100,
            'early_stopping_rounds' : 50 # 用earlystopping后训练时间更短，可以有效避免过拟合，得到的模型准确率更高。
           }

     |████████████████████████████████| 64.8MB 66kB/s 


## 每个商店单独训练模型

In [ ]:
# STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
STORES_IDS = ['WI_1','WI_2','WI_3']
END_TRAIN = 1941
############################################# Train Models ###############################################
for store_id in STORES_IDS: # 每个商店单独训练
    print('Train', store_id)
    grid_df, features_columns = get_data_by_store(store_id) # grid_df：74 features_columns：71

    ########################## 添加节假日特性 #############################
    ## CA1、TX2、TX3的特征的节假日效应类似，用同一个文件
    if (store_id=='CA_1') or (store_id=='TX_2') or (store_id=='TX_3'):
      calendar_win = pd.read_csv(fea_path + 'CA1_TX2_TX3_holidays.csv')
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16) 
      calendar_win['event_name_1_win'] = calendar_win['event_name_1_win'] 
      grid_df = grid_df.merge(calendar_win, on='d', how='left') # 小维度融入大维度按共有列进行merge

      features_columns = features_columns + ['event_name_1_win'] # 72
      if store_id=='CA1':
        cb_params = cb_params_common
      else:
        cb_params = cb_params_common
    ## CA2单独有自己的节假日效应
    elif store_id=='CA_2':
      calendar_win = pd.read_csv(fea_path + 'CA2_holidays.csv')
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)
      calendar_win['event_name_1_win'] = calendar_win['event_name_1_win']
      grid_df = grid_df.merge(calendar_win, on='d', how='left')

      features_columns = features_columns+['event_name_1_win']
      cb_params = cb_params_common
    ## TX1的节假日效应，假设所有节假日的前一天和后一天都有一定的影响
    elif store_id=='TX_1':
      calendar_df = pd.read_csv(datasets_path + 'calendar.csv')
      calendar_win = calendar_df[['event_name_2','event_name_1','d']]
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)

      # 添加特征
      event = calendar_df[['event_name_2','event_name_1']].fillna('') # 用''替换NAN(因为NAN为float类型)
      event_up_one = event.shift(-1).fillna('') # 上移之后，最后一行会出现NAN，需要再fillna('')一次
      event_before = pd.DataFrame()
      event_before['event_name_2'] = event_up_one['event_name_2'].apply(lambda x: x + '_before' if x != '' else x)
      event_before['event_name_1'] = event_up_one['event_name_1'].apply(lambda x: x + '_before' if x != '' else x)
      # 总结：对Series应用匿名函数时候，这一列必须是相同的数据类型，我要实现把节假日(字符串)改名字，这一列都必须要是str，所以把 NAN 填充为空字符串！
      # 这样，我就可以用匿名函数，返回的x是相同的数据类型。以后对含有空值的列处理其中的字符串元素的话，就要这么做，先填充，再apply匿名函数
      
      event_down_one = event.shift(1).fillna('') # 下移之后，第一行会出现NAN，需要再fillna('')一次
      event_after = pd.DataFrame()
      event_after['event_name_2'] = event_down_one['event_name_2'].apply(lambda x: x + '_after' if x != '' else x)
      event_after['event_name_1'] = event_down_one['event_name_1'].apply(lambda x: x + '_after' if x != '' else x)

      even_win = event_before + event_after # 注意只有都是字符串列也可以对dataframe相加。另外，'' + '' = ''

      even_win['event_name_2'] = even_win['event_name_2'].apply(lambda x: np.nan if x=='' else x)
      even_win['event_name_1'] = even_win['event_name_1'].apply(lambda x: np.nan if x=='' else x)

      # 去掉原来的节假日列，增加新的两列
      calendar_win.drop(columns=['event_name_1','event_name_2'],inplace=True) 
      calendar_win['event_name_1_win'] = even_win['event_name_1'] 
      calendar_win['event_name_2_win'] = even_win['event_name_2']
      calendar_win['d'] = calendar_win['d'].astype(np.int16) 

      # 融入grid_df中，74 + 2 = 76
      grid_df = grid_df.merge(calendar_win, on='d', how='left') # calendar_win：'d'、'event_name_1_win'、'event_name_2_win'这三列
      features_columns = features_columns + ['event_name_1_win','event_name_2_win'] # 71 + 2 = 73

      # 补充的节假日效应列
      calendar_win_1 = pd.read_csv(fea_path + 'CA1_TX2_TX3_holidays.csv')
      calendar_win_1.rename(columns = {'event_name_1_win':'event_name_1_win_1'},inplace=True)
      calendar_win_1['d'] = calendar_win_1['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)
      grid_df = grid_df.merge(calendar_win_1, on='d', how='left')
      features_columns = features_columns + ['event_name_1_win_1']

      # 设置模型参数
      cb_params = cb_params_common

    ## WI1的节假日效应，假设所有节假日的前两天和后一天都有一定的影响
    elif store_id=='WI_1':
      calendar_df = pd.read_csv(datasets_path + 'calendar.csv')
      calendar_win = calendar_df[['event_name_2','event_name_1','d']]
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)

      # 添加特征
      event = calendar_df[['event_name_2','event_name_1']].fillna('') 
      # 上移
      event_before1 = pd.DataFrame()
      event_up_one = event.shift(-1).fillna('') 
      event_before1['event_name_2'] = event_up_one['event_name_2'].apply(lambda x: x + '_before1' if x != '' else x)
      event_before1['event_name_1'] = event_up_one['event_name_1'].apply(lambda x: x + '_before1' if x != '' else x)
      event_before2 = pd.DataFrame()
      event_up_two = event.shift(-2).fillna('')   
      event_before2['event_name_2'] = event_up_two['event_name_2'].apply(lambda x: x + '_before2' if x != '' else x)
      event_before2['event_name_1'] = event_up_two['event_name_1'].apply(lambda x: x + '_before2' if x != '' else x)
      event_before = event_before1 + event_before2
      # 下移
      event_after = pd.DataFrame()
      event_down_one = event.shift(1).fillna('') 
      event_after['event_name_2'] = event_down_one['event_name_2'].apply(lambda x: x + '_after' if x != '' else x)
      event_after['event_name_1'] = event_down_one['event_name_1'].apply(lambda x: x + '_after' if x != '' else x)

      even_win = event_before + event_after 

      # 还原NAN
      even_win['event_name_2'] = even_win['event_name_2'].apply(lambda x: np.nan if x=='' else x)
      even_win['event_name_1'] = even_win['event_name_1'].apply(lambda x: np.nan if x=='' else x)
      # 去掉原来的节假日列，增加新的两列
      calendar_win.drop(columns=['event_name_1','event_name_2'],inplace=True) 
      calendar_win['event_name_1_win'] = even_win['event_name_1']
      calendar_win['event_name_2_win'] = even_win['event_name_2']
      calendar_win['d'] = calendar_win['d'].astype(np.int16) 

      # 融入grid_df中，74 + 2 = 76
      grid_df = grid_df.merge(calendar_win, on='d', how='left') # calendar_win：'d'、'event_name_1_win'、'event_name_2_win'这三列
      features_columns = features_columns + ['event_name_1_win','event_name_2_win'] # 71 + 2 = 73

      cb_params = cb_params_common

    ## 独自的节假日效应
    elif store_id=='WI_3':
      calendar_win = pd.read_csv(fea_path + 'WI_3_holidays.csv')
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)
      calendar_win['event_name_1_win'] = calendar_win['event_name_1_win']
      grid_df = grid_df.merge(calendar_win, on='d', how='left')
      features_columns = features_columns + ['event_name_1_win']
      cb_params = cb_params_common
    # WI2、CA3、CA4，这三个商店只考虑所有节假日的前两天的效应
    else:
      calendar_df = pd.read_csv(datasets_path + 'calendar.csv')
      calendar_win = calendar_df[['event_name_2','event_name_1','d']]
      # 添加特征
      event = calendar_df[['event_name_2','event_name_1']].fillna('') 
      # 上移
      event_before1 = pd.DataFrame()
      event_up_one = event.shift(-1).fillna('') 
      event_before1['event_name_2'] = event_up_one['event_name_2'].apply(lambda x: x + '_before1' if x != '' else x)
      event_before1['event_name_1'] = event_up_one['event_name_1'].apply(lambda x: x + '_before1' if x != '' else x)
      event_before2 = pd.DataFrame()
      event_up_two = event.shift(-2).fillna('')   
      event_before2['event_name_2'] = event_up_two['event_name_2'].apply(lambda x: x + '_before2' if x != '' else x)
      event_before2['event_name_1'] = event_up_two['event_name_1'].apply(lambda x: x + '_before2' if x != '' else x)
      event_win = event_before1 + event_before2

      calendar_win.drop(columns=['event_name_1','event_name_2'],inplace=True)
      # 转换数据类型
      calendar_win['event_name_1_win'] = event_win['event_name_1']
      calendar_win['event_name_2_win'] = event_win['event_name_2']
      calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)

      grid_df = grid_df.merge(calendar_win, on='d', how='left')
      features_columns = features_columns + ['event_name_1_win','event_name_2_win']

      cb_params = cb_params_common

    # 加入节假日特征之后的features_columns和grid_df
    print(features_columns) # CA1：72
    print(grid_df.shape) # CA1：(4873639, 75)
    ############## cb模型：cat_features必须为整数或字符串，实数值和NaN值应转换为字符串 ##############
    categorical_features_indices = ['item_id','dept_id','cat_id','snap_CA','snap_TX','snap_WI',
                      'event_name_1','event_type_1','event_name_2','event_type_2',
                      'event_name_1_win','event_name_2_win','event_name_1_win_1']
    categorical_features_indices = set(features_columns) & set(categorical_features_indices) # 取两个list的交集：set(list1) & set(list2)
    for i in categorical_features_indices:
      grid_df[i] = grid_df[i].astype('object') # 类别数据的类型要是object
      grid_df[i] = grid_df[i].fillna('no') # 注意：cb中不能出现空值，类别列也不可以

    train_mask = grid_df['d'] <= END_TRAIN - P_HORIZON # 1<= train <=1941-28 
    valid_mask = (grid_df['d'] > (END_TRAIN - P_HORIZON)) & (grid_df['d'] <= END_TRAIN) # 1941-28< valid <=1941
    preds_mask = grid_df['d'] > (END_TRAIN - 100) 

    # 封装cb的训练集和测试集
    train_pool = Pool(grid_df[train_mask][features_columns], grid_df[train_mask][TARGET], cat_features = categorical_features_indices) 
    valid_pool = Pool(grid_df[valid_mask][features_columns], grid_df[valid_mask][TARGET], cat_features = categorical_features_indices)

    ## 由于每个商店的节假日效应加的特征不一样，需要分开保存下来
    features_columns = pd.Series(features_columns) # 不转化为Series，会出现BUG：'list' object has no attribute 'to_pickle'
    features_columns.to_pickle(model_pkl_path +'{}_train_Fea(cb).pkl'.format(store_id)) # 存储训练模型时用的特征

    # 构造一个后面预测用的数据集 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col] # '_tmp_' 滞后 + rolling 比如：rolling_mean_tmp_14_60 去掉了12列
    grid_df = grid_df[keep_cols] # CA1：75 - 12 = 63 (390272, 63)  [1842, 1969] 128天
    grid_df.to_pickle(model_pkl_path + 'cb_test_{}.pkl'.format(store_id))

    seed_everything(SEED)
    # 训练模型
    cb_model = CatBoostRegressor(**cb_params) 
    cb_model.fit(train_pool, eval_set = valid_pool) 
    # 存储模型为二进制文件，下次读取的时候，速度更快
    model_name = model_pkl_path + 'cb_model_{}_v{}.bin'.format(store_id, str(VER))
    pickle.dump(cb_model, open(model_name, 'wb'))

    del grid_df, train_pool, valid_pool, cb_model
    gc.collect()

Train WI_1
['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'r

In [ ]:
### 查看某个商店由模型训练得到的特征重要度
model_bin = 'cb_model_{}_v{}.bin'.format('CA_1', str(VER))
if USE_AUX:
  model_path = Model_path + model_bin
  cb_model = pickle.load(open(model_path, 'rb'))
# 训练结束后，通过 model.feature_importances_ 属性，可以拿到这些特征的重要程度数据，特征的重要性程度可以帮助我们分析出一些有用的信息。
# 执行下面的代码，可以拿到特征重要程度的可视化结构。
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight') # 这个风格是白底带网格的
fea_ = cb_model.feature_importances_ # <class 'numpy.ndarray'>
fea_name = cb_model.feature_names_ # <class 'numpy.ndarray'>
plt.figure(figsize=(15,18))
plt.barh(fea_name, fea_) # 名字作为x，重要值作为y
plt.show()
# 使用matplotlib.pyplot.barh()函数来绘制水平型的条形图，注意：与bar()函数最大的不同是X轴和Y轴是颠倒过来的
# 选择使用不同的作图主题，使用 style.use() 函数即可

## 预测

In [ ]:
## 如果前面训练断开了，方便下面预测直接使用
CA1 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win']
CA1 = pd.Series(CA1) ## 转化为Series，才能用to_pickle()
CA1.to_pickle(model_pkl_path +'CA_1_train_Fea.pkl')

CA2 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win']
CA2 = pd.Series(CA2)
CA2.to_pickle(model_pkl_path +'CA_2_train_Fea.pkl')

CA3 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win', 'event_name_2_win']
CA3 = pd.Series(CA3)
CA3.to_pickle(model_pkl_path +'CA_3_train_Fea.pkl')

CA4 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win', 'event_name_2_win']
CA4 = pd.Series(CA4)
CA4.to_pickle(model_pkl_path +'CA_4_train_Fea.pkl')

TX1 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win', 'event_name_2_win', 'event_name_1_win_1']
TX1 = pd.Series(TX1)
TX1.to_pickle(model_pkl_path +'TX_1_train_Fea.pkl')

TX2 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win']
TX2 = pd.Series(TX2)
TX2.to_pickle(model_pkl_path +'TX_2_train_Fea.pkl')

TX3 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win']
TX3 = pd.Series(TX3)
TX3.to_pickle(model_pkl_path +'TX_3_train_Fea.pkl')

WI1 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win', 'event_name_2_win']
WI1 = pd.Series(WI1)
WI1.to_pickle(model_pkl_path +'WI_1_train_Fea.pkl')

WI2 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win', 'event_name_2_win']
WI2 = pd.Series(WI2)
WI2.to_pickle(model_pkl_path +'WI_2_train_Fea.pkl')

WI3 = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'event_name_1_win']
WI3 = pd.Series(WI3)
WI3.to_pickle(model_pkl_path +'WI_3_train_Fea.pkl')

In [ ]:
USE_AUX = True
END_TRAIN = 1941
all_preds = pd.DataFrame()
STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
base_test = get_base_test() 
# 3902720 rows × 66 columns 62 + 4（event_name_1_win、store_id、event_name_2_win、event_name_1_win_1）
# 记住：后面这个特征是新加的，原来只有62 + 12(lag+rolling) = 72
main_time = time.time()

for PREDICT_DAY in range(1,29):    
  print('Predict | Day:', PREDICT_DAY)
  start_time = time.time() 
  grid_df = base_test.copy() 
  grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1) 

  for store_id in STORES_IDS:
    model_bin = 'cb_model_{}_v{}.bin'.format(store_id, str(VER))
    if USE_AUX:
      model_path = model_pkl_path + model_bin
    cb_model = pickle.load(open(model_path, 'rb')) # 读取训练好的模型
    MODEL_FEATURES = pd.read_pickle(model_pkl_path +'{}_train_Fea(cb).pkl'.format(store_id)).tolist() 
    categorical_features_indices = ['item_id','dept_id','cat_id','snap_CA','snap_TX','snap_WI',
                      'event_name_1','event_type_1','event_name_2','event_type_2',
                      'event_name_1_win','event_name_2_win','event_name_1_win_1']
    categorical_features_indices = set(MODEL_FEATURES) & set(categorical_features_indices) 
    for i in categorical_features_indices:
      grid_df[i] = grid_df[i].astype('object')
      grid_df[i] = grid_df[i].fillna('no') 
    ## 某一天
    day_mask = base_test['d'] == (END_TRAIN + PREDICT_DAY) # 1942、1943、...、1969
    ## 某个店
    store_mask = base_test['store_id'] == store_id
    ## 某一天某个店
    mask = (day_mask) & (store_mask)
    ## 预测某商店在 END_TRAIN + PREDICT_DAY 这一天商品的销量
    base_test[TARGET][mask] = cb_model.predict(grid_df[mask][MODEL_FEATURES])
    ## 这里当时出现了一个BUG，说训练集和验证集的类别变量跟测试集不一致，然后我查看一下grid_df.info()发现
    ## 里面的节假日效应在构造测试集的时候没有转化为"category"，然后我就在get_base_test()里面把它们转化为类别变量，这样就可以预测了

  temp_df = base_test[day_mask][['id',TARGET]]
  temp_df.columns = ['id', 'F' + str(PREDICT_DAY)]
  if 'id' in list(all_preds):
    all_preds = all_preds.merge(temp_df, on=['id'], how='left')
  else:
    all_preds = temp_df.copy()
      
  print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
          ' %0.2f min total |' % ((time.time() - main_time) / 60),
          ' %0.2f day sales |' % (temp_df['F' + str(PREDICT_DAY)].sum()))
  del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.to_csv(pred_path + 'per_store_pred(cb).csv',index=False)

# 用于生成提交预测的文件
submission = pd.read_csv(datasets_path + 'sample_submission.csv')[['id']] # [[]] 返回DataFrame [] 返回Series
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(sub_path + 'submission_by_stores(cb).csv'), index=False) 

Predict | Day: 1
##########  1.57 min round |  1.57 min total |  38796.23 day sales |
Predict | Day: 2
##########  1.45 min round |  3.02 min total |  36175.43 day sales |
Predict | Day: 3
##########  1.45 min round |  4.47 min total |  35815.20 day sales |
Predict | Day: 4
##########  1.44 min round |  5.91 min total |  36006.28 day sales |
Predict | Day: 5
##########  1.45 min round |  7.36 min total |  41032.78 day sales |
Predict | Day: 6
##########  1.44 min round |  8.80 min total |  49654.95 day sales |
Predict | Day: 7
##########  1.42 min round |  10.22 min total |  49233.42 day sales |
Predict | Day: 8
##########  1.43 min round |  11.66 min total |  40840.35 day sales |
Predict | Day: 9
##########  1.44 min round |  13.09 min total |  39064.69 day sales |
Predict | Day: 10
##########  1.43 min round |  14.52 min total |  42542.58 day sales |
Predict | Day: 11
##########  1.43 min round |  15.95 min total |  42915.36 day sales |
Predict | Day: 12
##########  1.44 min round | 

## 某个商店进行贝叶斯调参

In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
def rmse(y, y_pred):
  return np.sqrt(np.mean(np.square(y - y_pred)))

store_id = ['CA_1']
TARGET = 'sales'
day = 1941
grid_df, features_columns = get_data_by_store(store_id)
df = grid_df[grid_df['d'] <= day]
del grid_df, features_columns

MODEL_FEATURES = pd.read_pickle(model_pkl_path +'{}_train_Fea(cb).pkl'.format(store_id)).tolist() 

# 1000到1941-28-28 作为训练集
train_mask = (df['d'] >= 1000) & (df['d'] <= day-28-28)
# 1941-28-28到1941-28 作为测试集
valid_mask = (df['d'] > (day-28-28)) & (df['d'] <= (day-28))

# 读取商店自己的训练特征
MODEL_FEATURES = pd.read_pickle(model_pkl_path +'{}_train_Fea(cb).pkl'.format(store_id)).tolist() 

categorical_features_indices = ['item_id','dept_id','cat_id','snap_CA','snap_TX','snap_WI',
                  'event_name_1','event_type_1','event_name_2','event_type_2',
                  'event_name_1_win','event_name_2_win','event_name_1_win_1']
categorical_features_indices = set(MODEL_FEATURES) & set(categorical_features_indices) 
for i in categorical_features_indices:
  grid_df[i] = grid_df[i].astype('object')
  grid_df[i] = grid_df[i].fillna('no')

train_pool = Pool(grid_df[train_mask][MODEL_FEATURES], grid_df[train_mask][TARGET], cat_features = categorical_features_indices) 
valid_pool = Pool(grid_df[valid_mask][MODEL_FEATURES], grid_df[valid_mask][TARGET], cat_features = categorical_features_indices)

# 1914到1941 作为测试集，用训练的模型预测它的"sales"，再与真实的"sales"计算RMSE
test_df = df[df['d']>(day-28)].reset_index(drop=True)

# 定义黑盒函数
def rf_cv(iterations, l2_leaf_reg, learning_rate, subsample, max_depth, max_bin, min_data_in_leaf):

  model = CatBoostRegressor(
                  iterations=int(iterations), # 最大树的个数，默认1000，可以设置的大一些，1000~2000
                  learning_rate=learning_rate, # 0.01~0.1，范围：[e^{-7}, 1]
                  l2_leaf_reg=int(l2_leaf_reg), # 损失函数权重的系数，默认为3，范围：[1,10]
                  subsample=subsample, # bagging的采样率，当bootstrap_type为Poisson或Bernoulli时使用，默认0.66
                  max_depth=int(max_depth), # 在大多数情况下，最佳深度范围从4到10。建议使用6到10之间的值。
                  boosting_type='Plain', # 相当于lgb的GBDT
                  one_hot_max_size=2, # 如果feature包含的不同值的数目超过了指定值，将feature转化为float，默认False，范围：[0,25]
                  max_bin=int(max_bin), # 同border_count,连续值离散化时bin的最大个数,过大容易过拟合
                  objective='RMSE', # 目标函数
                  eval_metric='RMSE', # 评估指标
                  random_seed=42, # 固定
                  nan_mode='Max', # 处理输入数据中缺失值的方法，包括Forbidden(禁止存在缺失)，Min(用最小值补)，Max(用最大值补)，默认Min。
                  min_data_in_leaf=int(min_data_in_leaf), # 叶子节点最小样本数，default=1
                  boost_from_average= True, # 调整初始的分数为标签的均值，加速模型训练的收敛速度，仅用于回归任务
                  verbose=False # 是否显示详细信息，default=1为输出进度条记录
                )

    cb_model_Bayes = model.fit(train_pool, eval_set = valid_pool) # 训练过程中，绘制，度量值，所用时间等
    test_df['preds'] = cb_model_By.predict(test_df[MODEL_FEATURES])
    base_score = rmse(test_df['sales'], test_df['preds'])

    return -base_score

# 给定超参数搜索空间
rf_bo = BayesianOptimization(rf_cv, 
                    {
                      'iterations': (1500, 2000),
                      'l2_leaf_reg': (3, 10),
                      'learning_rate': (0.01, 0.1),
                      'subsample': (0.6, 1),
                      'max_depth':(6, 10),
                      'min_data_in_leaf':(2**10-1,2**15-1),
                      'max_bin':(80,150)   
                    }
                ) 
opt.maximize(n_iter = 20) # 最大化黑盒函数，迭代100次

rf_bo.max # 返回黑盒函数值最大的超参数

|   iter    |  target   | baggin... | featur... | lambda_l2 | learni... |  max_bin  | min_da... | n_esti... | num_le... | sub_fe... |  sub_row  | subsample | subsam... | tweedi... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -2.005    |  0.4722   |  0.4987   |  0.0196   |  0.03115  |  87.66    |  6.255e+0 |  1.438e+0 |  7.805e+0 |  0.7139   |  0.7916   |  0.5068   |  0.9146   |  1.496    |
|  2        | -2.31     |  4.875    |  0.4995   |  0.138    |  0.4799   |  145.5    |  4.86e+03 |  1.511e+0 |  1.923e+0 |  0.7036   |  0.8444   |  0.776    |  1.481    |  1.052    |
|  3        | -2.142    |  0.554    |  0.4238   |  0.09402  |  0.8591   |  118.5    |  2.158e+0 |  1.074e+0 |  2.461e+0 |  0.6854   |  0.68     |  0.5021   |  4.48     |  1.46     |
|  4        | -2.125    |  2.975    |  0.5173   |  0.1388   |  0.1773   |  95.7     |  3.5